# Machine Learning Dunant 1

**Doel:** We bouwen een eenvoudig, reproduceerbaar ML‑proces om dagelijks gasverbruik en het beoveld‑verbruik tijdens wintermaanden te voorspellen. Het model wordt getraind op een "goed" referentiejaar zodat we daarna afwijkingen in andere jaren kunnen signaleren (bv. installatieproblemen of alarmen).

#### Waarom deze aanpak

- Praktisch: inzicht in hoe temperatuur, weekdag en vakanties het verbruik beïnvloeden.  
- Diagnostisch: voorspellingen gebruiken om afwijkend gedrag van de installatie te detecteren.  
- Reproduceerbaar: volledige pipeline (feature‑engineering → imputatie → training → opslag model → predicties).

#### Data (kort) 

- Features: weekdag (cyclisch), weeknummer (cyclisch), seizoen, weekend/vakantie flag, temp_avg, temp_min, temp_max.  
- Targets: gasverbruik, beo_opwekking (beo‑veld).  
- We gebruiken dagelijkse aggregaties; ontbrekende waardes en outliers worden expliciet behandeld.

#### Leerdoelen (wat je na de workshop kunt)

1. Data voorbereiden en tijdskenmerken cyclisch coderen.  
2. Missende waarden en outliers herkennen en behandelen.  
3. Eenvoudige preprocessing‑pipeline (imputatie, encoding, scaling) bouwen.  
4. Random Forest trainen, tunen en opslaan als productiemodel.  
5. Nieuwe periode (winter 24/25) voorbereiden en voorspellingen maken om installatiegedrag te evalueren.

#### Stappen (kort)

1. Data inlezen en verkennen  
2. Feature engineering (incl. cyclische encoding)  
3. Imputatie en outlierverwijdering  
4. Train/test split en baseline modellen vergelijken  
5. Random Forest finetunen met TimeSeriesSplit  
6. Final model trainen, opslaan en gebruiken voor 24/25‑voorspellingen  
7. Interpretatie van resultaten en vervolgstappen

#### Vereisten

- Basis Python/pandas ervaring  
- Geïnstalleerde libs: scikit‑learn, pandas, numpy, matplotlib/seaborn, joblib

#### Verwachte output

- Notebook met volledige pipeline, opgeslagen Random Forest modellen en CSV met voorspellingen voor 24/25 plus korte interpretatie van afwijkingen.

Kort en concreet: we trainen op een representatief goed jaar, bewaren het beste model en gebruiken dat model om in de winter 24/25 afwijkingen in verbruik te detecteren.

In [ ]:
import pandas as pd
import holidays
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.tsa.seasonal import STL
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

## Stap 1: Dataverzameling

1. Dataframe opstellen met alle dagen van 2023
2. Datum feature kolommen toevoegen
3. Weerdata toevoegen
4. y data toevoegen

### 1. Dataframe opstellen met alle dagen van 2023

In [ ]:
ml_df = pd.DataFrame({'datum': pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')})
ml_df.head()

### 2. Datumfeatures toevoegen

1. Dag van de week (cijfers met start op maandag)
2. Weeknummer (cijfer)
3. seizoen
4. IsWeekdag
5. IsVakantiedag

In [ ]:
ml_df['weekdag'] = ml_df['datum'].dt.day_of_week
ml_df['weeknummer'] = ml_df['datum'].dt.isocalendar().week
ml_df['seizoen'] = pd.cut(ml_df['datum'].dt.month,
                         bins=[0, 3, 6, 9, 12],
                         labels=['Winter', 'Lente', 'Zomer', 'Herfst'])
ml_df['is_weekend'] = (ml_df['datum'].dt.dayofweek >= 5).astype(int)

be_holidays = holidays.BE(years=ml_df['datum'].dt.year.unique().tolist())
ml_df['is_vakantiedag'] = ml_df['datum'].isin(be_holidays).astype(int)

ml_df.head()

### 3. Weerdata toevoegen

1. Zoek het open dataplatform van het KMI
2. Download de data voor 2023 (AWS 1 dag)
3. Voeg de weerdata toe aan de dataframe
4. Het beste weerstation heeft geopunten (50.98 3.816)

In [ ]:
# Laad de weerdata in

In [ ]:
# Filter op de juiste geopunten

We houden volgende kolommen over: timestamp, temp_avg, temp_max, temp_min

In [ ]:
# Filter de juiste kolommen

In [ ]:
# Hernoem de kolom timestamp naar datum en zet dit om naar een datetime object en normaliseer het

In [ ]:
# Merge (left) de data met de ml_df

In [ ]:
# 1. laad de gasteller in (data/gasteller.txt), seperator is een tap (sep='\t") en laat de header (titel kolomen) weg
gasteller =

# 2. Hernoem de kolommen naar datum en gastellerstand en verwijder de laatste kolom indien nodig


# 3. Zet de datumkolom om naar een datetime object


# 4. Filter de dataset zoals gezien in 2_gasketel_rendement


In [ ]:
# Zet de datum goed, normaliseer.

# De verbruiken worden om 12u snachts opgenomen, dus het verbruik is eigenlijk van de dag ervoor. Laten we een .shift(-1) uitvoeren op de juiste kolom!

# Voeg het verbruik toe aan de ml_df


In [ ]:
# lees de calorieteller uit van het beoveld
cal_beo = 

In [ ]:
# Voer de zelfde bewerkingen uit, zet om naar Kwh, normaliseer en shift de opwekking_warm met -1


In [ ]:
# voeg de kolom toe aan ml_df


## Stap 2: Datavisualisatie

In [ ]:
df = ml_df.copy()

sns.set_theme(style='whitegrid')
fig, axes = plt.subplots(3, 2, figsize=(15, 18))
ax = axes.flatten()

# 1) Tijdreeks gasverbruik + 7-dagen rolling gemiddelde
g1 = df.dropna(subset=['gasverbruik'])
ax[0].plot(g1['datum'], g1['gasverbruik'], color='#0078D4', alpha=0.6, label='Dagelijks')
ax[0].plot(g1['datum'], g1['gasverbruik'].rolling(7, min_periods=1).mean(), color='orange', linewidth=2, label='7d MA')
ax2 = ax[0].twinx()
ax2.plot(g1['datum'], g1['temp_avg'], color='red', label='Temp AVG')
ax2.set_ylabel('Temp AVG (°C)', color='red')
ax2.tick_params(axis='y', colors='red')
ax[0].set_title('Gasverbruik - tijdreeks')
ax[0].set_ylabel('kWh')
ax[0].legend()

# 2) Scatter temp_avg vs gasverbruik (met regressielijn)
g2 = df.dropna(subset=['temp_avg','gasverbruik'])
sns.regplot(data=g2, x='temp_avg', y='gasverbruik', scatter_kws={'alpha':0.6}, line_kws={'color':'red'}, ax=ax[1])
ax[1].set_title('Temp_avg vs Gasverbruik')
ax[1].set_xlabel('Gem. buitentemp (°C)')

# 3) Boxplot gasverbruik per weekdag
g3 = df.dropna(subset=['weekdag','gasverbruik'])
sns.boxplot(data=g3, x='weekdag', y='gasverbruik', ax=ax[2], palette='Blues')
ax[2].set_title('Gasverbruik per weekdag')
ax[2].set_xlabel('Weekdag (0=maandag)')
ax[2].set_ylabel('kWh')

# 4) Correlatie heatmap van relevante features
cols = ['gasverbruik','temp_avg','temp_min','temp_max','is_weekend','is_vakantiedag','beo_opwekking']
corr = df[cols].corr()
sns.heatmap(corr, annot=True, fmt='.2f', cmap='vlag', center=0, ax=ax[3])
ax[3].set_title('Correlatiematrix')

# 5) Beo_opwekking tijdreeks + 7-dagen MA
g5 = df.dropna(subset=['beo_opwekking'])
ax[4].plot(g5['datum'], g5['beo_opwekking'], color='#2ca02c', alpha=0.7, label='Beo opwekking')
ax[4].plot(g5['datum'], g5['beo_opwekking'].rolling(7, min_periods=1).mean(), color='orange', linewidth=2, label='7d MA')
ax4 = ax[4].twinx()
ax4.plot(g1['datum'], g1['temp_avg'], color='red', label='Temp AVG')
ax4.set_ylabel('Temp AVG (°C)', color='red')
ax4.tick_params(axis='y', colors='red')
ax[4].set_title('Beo opwekking - tijdreeks')
ax[4].set_ylabel('Opwekking (kWh)')
ax[4].set_xlabel('Datum')
ax[4].legend()

# 6) Temp_avg vs Beo_opwekking (met regressielijn)
g6 = df.dropna(subset=['temp_avg','beo_opwekking'])
sns.regplot(data=g6, x='temp_avg', y='beo_opwekking', scatter_kws={'alpha':0.6}, line_kws={'color':'red'}, ax=ax[5])
ax[5].set_title('Temp_avg vs Beo Opwekking')
ax[5].set_xlabel('Gem. buitentemp (°C)')
ax[5].set_ylabel('Beo opwekking (kWh)')

plt.tight_layout()
plt.show()

## Stap 3: Verwijderen van outliers

Deze stap is optioneel, maar sterk aan te raden. Outliers kunnen een negatief effect hebben op de prestaties van ML modellen.

### Outliers gasverbruik verwijderen

In tijdreeksdata outliers verwijderen kan een moeilijke taak zijn, je kan deze manueel gaan verwijderen, echter we zien dat het beoveld nog steeds niet optimaal werkte, maar we moeten het doen met wat we hebben. Een mogelijkheid om echte outliers slim te verwijderen is door de som te nemen van het gas en beoveld verbruik, daarbij de rolling gemiddelde te nemen, en dan de rijen die er erg uitschieten te verwijderen uit de data. Deze techniek kan je hieronder terugvinden.

In [ ]:
# Stap 0: Repareer eerst de gaten in je ruwe data (Interpolatie)
# limit=5 betekent: vul gaten van max 5 dagen op. Als een gat groter is, is het misschien beter om het leeg te laten.
ml_df['gasverbruik'] = ml_df['gasverbruik'].interpolate(method='linear', limit=10)
ml_df['beo_opwekking'] = ml_df['beo_opwekking'].interpolate(method='linear', limit=10)

In [ ]:
# Stap 0.1: controleer of de data compleet is
time_df = ml_df.copy()

time_df = time_df.sort_values('datum')
time_df['tijdsprong'] = time_df['datum'].diff()

gaten = time_df[time_df['tijdsprong'] > pd.Timedelta('1D')]

print('Gevonden gaten:')
print(gaten[['datum', 'tijdsprong']])

In [ ]:
# Stap 1: Totaal verbruik toevoegen


In [ ]:
# Stap 2: Bereken rolling statistics (bijv. per 7 dagen om weekpatronen mee te pakken)
# center=True is belangrijk: zo vergelijk je een dag met de dagen er vlak voor én erna.
window_size = 7
ml_df['rolling_mean'] = ml_df['totaal_verbruik'].rolling(window=window_size, center=True).mean()
# Doe het zelfde voor de standaard afwijking (std)

In [ ]:
# Stap 3: Definieer outliers
# Een punt is een outlier als het meer dan X keer de standaardafwijking afwijkt van het lokale gemiddelde.
# 3 is standaard (streng), 2 is wat losser (pakt meer outliers).
sigma_threshold = 2

ml_df['is_outlier'] = (
    ml_df['totaal_verbruik'] > (ml_df['rolling_mean'] + sigma_threshold * ml_df['rolling_std'])
) | (
    ml_df['totaal_verbruik'] < (ml_df['rolling_mean'] - sigma_threshold * ml_df['rolling_std'])
)

In [ ]:
# Stap 4: Inspectie (belangrijk!)
# Plot de resultaten voordat je verwijdert, om te zien of je geen koude dips weggooit.
plt.figure(figsize=(12, 6))
plt.plot(ml_df['datum'], ml_df['totaal_verbruik'], label='Totaal Verbruik', alpha=0.6)
plt.plot(ml_df['datum'], ml_df['rolling_mean'], label='Rolling Mean (Trend)', color='green')
# Teken de outliers als rode punten
outliers = ml_df[ml_df['is_outlier']]
plt.scatter(outliers['datum'], outliers['totaal_verbruik'], color='red', label='Gedetecteerde Outlier', zorder=5)
plt.legend()
plt.title("Detectie van uitschieters op basis van lokaal gemiddelde")
plt.show()

We zien dat we 1 grote piek hebben, die niet wordt gededecteerd, daarnaast zien we ook dat de hele zomerperiode weinig nut heeft voor ons model. Mogelijke oplossing, data weg halen van juni tot oktober en de dagen ronde de piek manueel verwijderen.

In [ ]:
# Verwijder data van juni (6) tot en met oktober (10)
# Het teken '~' betekent 'NIET', dus: behoud alles wat NIET in die maanden valt.
ml_df = ml_df[~ml_df['datum'].dt.month.between(6, 10)]

In [ ]:
# Stap 4: Inspectie (belangrijk!)
# Plot de resultaten voordat je verwijdert, om te zien of je geen koude dips weggooit.
plt.figure(figsize=(12, 6))
plt.plot(ml_df['datum'], ml_df['totaal_verbruik'], label='Totaal Verbruik', alpha=0.6)
plt.plot(ml_df['datum'], ml_df['rolling_mean'], label='Rolling Mean (Trend)', color='green')
plt.show()

In [ ]:
# Verwijder rijen met meer dan 700 kwh verbruik op een dag


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(ml_df['datum'], ml_df['totaal_verbruik'], label='Totaal Verbruik', alpha=0.6)
plt.plot(ml_df['datum'], ml_df['rolling_mean'], label='Rolling Mean (Trend)', color='green')
plt.show()

## Stap 4: Verdelen van training en testdata

Weekdag -> tijdsvariabele en zetten we om naar cyclische waarde via sin en cos transformatie.  
weeknummer -> tijdsvariabele en zetten we om naar cyclische waarde via sin en cos transformatie.  
Seizoen -> categorische variabele en zetten we om via one-hot encoding.  
is_weekend -> binaire variabele (0 of 1)  
is_vakantiedag -> binaire variabele (0 of 1)  
temp_avg -> numerieke variabele  
temp_min -> numerieke variabele  
temp_max -> numerieke variabele  
gasverbruik -> target variabele (numeriek)  
beo_opwekking -> target variabele (numeriek)  

### Cyclische codering van tijdsvariabelen (weekdag_sin / weekdag_cos, week_sin / week_cos)

Waarom?
- Tijdgegevens zoals weekdag en weeknummer zijn cyclisch: maandag (0) en zondag (6) liggen dicht bij elkaar, maar numerieke encoding (`0..6`) negeert dat.
- Sin / cos-transformatie behoudt de cyclische structuur en geeft continue, vloeiende representaties die modellen makkelijk kunnen gebruiken.

Formule
- sin = sin(2π * value / period)
- cos = cos(2π * value / period)

Voorbeelden (gebruik in de notebook)
- Weekdag (0=maandag … 6=zondag), period = 7:
  ml_df['weekdag_sin'] = np.sin(2 * np.pi * ml_df['weekdag'] / 7)  
  ml_df['weekdag_cos'] = np.cos(2 * np.pi * ml_df['weekdag'] / 7)

- Weeknummer (ISO week 1–52/53). Vaak gebruik je 52 als period:
  ml_df['week_sin'] = np.sin(2 * np.pi * (ml_df['weeknummer'] - 1) / 52)  
  ml_df['week_cos'] = np.cos(2 * np.pi * (ml_df['weeknummer'] - 1) / 52)

Praktische tips
- Gebruik exact dezelfde encoding bij training en inference.  
- Sin/cos liggen in [-1, 1], meestal geen extra scaling nodig.  
- Als je weeknummers 53 krijgt, kies period = 53 of voorkom randen door te normaliseren naar jaarfractie.  
- Voor interpretatie kun je hoek = atan2(sin, cos) gebruiken om de oorspronkelijke fase terug te winnen.

In [ ]:
# gebruik bestaande datumkolom


# weeknummer (1-52/53) naar cyclisch (zorg dat je 52 of 53 kiest afhankelijk van dataset)


In [ ]:
# prepare features / targets, imputation + encoding + scaling pipeline, split and train
features = ['weekdag_sin','weekdag_cos','week_sin','week_cos','seizoen','is_weekend','is_vakantiedag','temp_avg','temp_max','temp_min']
targets = ['gasverbruik','beo_opwekking']

X = ml_df[features].copy()
y = ml_df[targets].copy()

In [ ]:
# handle missing targets: drop rows with missing target values (preferred)
mask = y.notna().all(axis=1)
X = X.loc[mask].reset_index(drop=True)
y = y.loc[mask].reset_index(drop=True)

In [ ]:
# column groups
num_feats = ['temp_avg','temp_min','temp_max']
cat_feats = ['seizoen']
passthrough = ['weekdag_sin','weekdag_cos','week_sin','week_cos','is_weekend','is_vakantiedag']

In [ ]:
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='NA')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
])

preproc = ColumnTransformer([
    ('num', num_pipeline, num_feats),
    ('cat', cat_pipeline, cat_feats)
], remainder='passthrough')

In [ ]:
pipelines = {
    'Linear': Pipeline([('pre', preproc), ('model', LinearRegression())]),
    'SVR': Pipeline([('pre', preproc), ('model', SVR(kernel='rbf', C=1.0))]),
    'RandomForest': Pipeline([('pre', preproc), ('model', RandomForestRegressor(n_estimators=100, random_state=42))])
}

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
# train per target and collect metrics
results = []
for tgt in targets:
    ytr = y_train[tgt].values
    yte = y_test[tgt].values
    for name, pipe in pipelines.items():
        pipe.fit(X_train, ytr)
        preds = pipe.predict(X_test)
        mae = mean_absolute_error(yte, preds)
        rmse = np.sqrt(mean_squared_error(yte, preds))  # vervangt squared=False
        results.append({'target': tgt, 'model': name, 'MAE': mae, 'RMSE': rmse})

results_df = pd.DataFrame(results).sort_values(['target','RMSE']).reset_index(drop=True)
print(results_df)

### Verder gaan met Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# 1. Maak een lege dictionary om je modellen in te bewaren
best_models = {} 

rf_pipe = Pipeline([('pre', preproc), ('model', RandomForestRegressor(random_state=42))])

param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [None, 10, 20],
    'model__max_features': ['sqrt', 0.5]
}

tscv = TimeSeriesSplit(n_splits=3)
results = []

for tgt in targets:
    print(f"Start training voor target: {tgt}")
    ytr = y_train[tgt].values
    yte = y_test[tgt].values

    gs = GridSearchCV(rf_pipe, param_grid, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
    gs.fit(X_train, ytr)

    best = gs.best_estimator_
    
    # 2. HIER SLAAN WE HET MODEL OP
    best_models[tgt] = best
    
    preds = best.predict(X_test)
    mae = mean_absolute_error(yte, preds)

    results.append({'target': tgt,
                    'model': 'RandomForest',
                    'best_params': gs.best_params_,
                    'MAE': mae
                    })

    # (Feature importance code blijft hetzelfde...)
    try:
        feat_names = best.named_steps['pre'].get_feature_names_out(X_train.columns)
    except Exception:
        feat_names = np.array([f'f{i}' for i in range(best.named_steps['pre'].transform(X_train.iloc[:1]).shape[1])])

    importances = best.named_steps['model'].feature_importances_
    fi = pd.Series(importances, index=feat_names).sort_values(ascending=False).head(15)
    print(f"Top features voor {tgt}:\n", fi.to_string())

results_df = pd.DataFrame(results).sort_values(['target','MAE']).reset_index(drop=True)
print("\nSamenvatting:\n", results_df)

## Maken van voorspelling voor 24-25

De winter van 24/25 was een dubbelle winter, de eerste helft verliep alles vrij goed, de 2de helft lag de warmtepomp vaak in alarm en behaalde ze haar target niet. Laten we zien of we dit kunnen voorspellen!

### Stap 1: Dataframe maken

We maken een dataframe van 2024-10-01 tot 2025-05-01.

In [ ]:
# Maak een dataframe voor de winter van 24-25


In [ ]:
# Voeg de datumfeatures toe

# Voeg de sin en cos toe voor week en weekdag
# gebruik bestaande datumkolom

# weeknummer (1-52/53) naar cyclisch (zorg dat je 52 of 53 kiest afhankelijk van dataset)


In [ ]:
# Verwerk weerdata, gedownload van opendata kmi en verwerk dit met de juiste kolommen


In [ ]:
# Voeg de data toe aan ml_2425


In [ ]:
# Voeg het gasverbruik toe voor 2425


In [ ]:
# Voeg beoveld gegevens toe voor 2425


Maak voorspellingen voor gasverbruik en beo_opwekking

In [ ]:
# 3. Zorg dat de kolommen in exact dezelfde volgorde staan als tijdens de training
features = ['weekdag_sin','weekdag_cos','week_sin','week_cos','seizoen',
            'is_weekend','is_vakantiedag','temp_avg','temp_max','temp_min']

# Selecteer de X voor 2025
X_2025 = ml_2425[features].copy()

In [ ]:
# Voorspel gasverbruik met het gas-model
# Het model (pipeline) regelt zelf de scaling/encoding, dus we kunnen X_2025 direct invoeren
ml_2425['pred_gas'] = best_models['gasverbruik'].predict(X_2025)

# Voorspel beo veld met het beo-model
ml_2425['pred_beo'] = best_models['beo_opwekking'].predict(X_2025)

# Resultaat bekijken
print(ml_2425[['datum', 'pred_gas', 'pred_beo']].head())

# Eventueel visualiseren
plt.figure(figsize=(15, 6))
plt.plot(ml_2425['datum'], ml_2425['pred_gas'], label='Voorspeld Gas', alpha=0.7)
plt.plot(ml_2425['datum'], ml_2425['pred_beo'], label='Voorspeld Beo', alpha=0.7)
plt.plot(ml_2425['datum'], ml_2425['gasverbruik'], label='Werkelijk verbruik', alpha=0.3)
plt.plot(ml_2425['datum'], ml_2425['beo_opwekking'], label='Werkelijke beo', alpha=0.3)
plt.title("Voorspelling verbruik winter 24- 25")
plt.legend()
plt.show()